In [ ]:
install.packages('nycflights13')

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)



In [2]:
options(repr.plot.width=8, repr.plot.height=5)
library(tidyverse)
library(nycflights13)

── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.2     ✔ readr     2.1.4
✔ forcats   1.0.0     ✔ stringr   1.5.0
✔ ggplot2   3.4.3     ✔ tibble    3.2.1
✔ lubridate 1.9.2     ✔ tidyr     1.3.0
✔ purrr     1.0.2     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors


# Lecture 5: Tidy data, missing data

<div style="border: 1px double black; padding: 10px; margin: 10px">

**After today's lecture you will:**
* Gain more experience creating and working with tidy data
* Learn about how R handles missing data.
    
This lecture note corresponds to Chapters 6 and 20 of your book. 
</div>


    




## A usage of `pivot_wider()`
Let's revisit the `gapminder` dataset that we first saw last lecture:

In [ ]:
install.packages('gapminder')

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)



In [4]:
library(gapminder)
print(gapminder)

# A tibble: 1,704 × 6
   country     continent  year lifeExp      pop gdpPercap
   <fct>       <fct>     <int>   <dbl>    <int>     <dbl>
 1 Afghanistan Asia       1952    28.8  8425333      779.
 2 Afghanistan Asia       1957    30.3  9240934      821.
 3 Afghanistan Asia       1962    32.0 10267083      853.
 4 Afghanistan Asia       1967    34.0 11537966      836.
 5 Afghanistan Asia       1972    36.1 13079460      740.
 6 Afghanistan Asia       1977    38.4 14880372      786.
 7 Afghanistan Asia       1982    39.9 12881816      978.
 8 Afghanistan Asia       1987    40.8 13867957      852.
 9 Afghanistan Asia       1992    41.7 16317921      649.
10 Afghanistan Asia       1997    41.8 22227415      635.
# ℹ 1,694 more rows


## Missing data in R
The `gapminder` appears very nice, it only contains data sets with no missing observations. But! The raw data looks like this:

In [5]:
gapminder_unfiltered  %>% print

# A tibble: 3,313 × 6
   country     continent  year lifeExp      pop gdpPercap
   <fct>       <fct>     <int>   <dbl>    <int>     <dbl>
 1 Afghanistan Asia       1952    28.8  8425333      779.
 2 Afghanistan Asia       1957    30.3  9240934      821.
 3 Afghanistan Asia       1962    32.0 10267083      853.
 4 Afghanistan Asia       1967    34.0 11537966      836.
 5 Afghanistan Asia       1972    36.1 13079460      740.
 6 Afghanistan Asia       1977    38.4 14880372      786.
 7 Afghanistan Asia       1982    39.9 12881816      978.
 8 Afghanistan Asia       1987    40.8 13867957      852.
 9 Afghanistan Asia       1992    41.7 16317921      649.
10 Afghanistan Asia       1997    41.8 22227415      635.
# ℹ 3,303 more rows


### What happens when we reshape the "unfiltered" data?

In [6]:
# pivot unfiltered data wider


# A tibble: 187 × 59
   country `1952` `1957` `1962` `1967` `1972` `1977` `1982` `1987` `1992` `1997`
   <fct>    <dbl>  <dbl>  <dbl>  <dbl>  <dbl>  <dbl>  <dbl>  <dbl>  <dbl>  <dbl>
 1 Afghan…   779.   821.   853.   836.   740.   786.   978.   852.   649.   635.
 2 Albania  1601.  1942.  2313.  2760.  3313.  3533.  3631.  3739.  2497.  3193.
 3 Algeria  2449.  3014.  2551.  3247.  4183.  4910.  5745.  5681.  5023.  4797.
 4 Angola   3521.  3828.  4269.  5523.  5473.  3009.  2757.  2430.  2628.  2277.
 5 Argent…  5911.  6857.  7133.  8053.  9443. 10079.  8998.  9140.  9308. 10967.
 6 Armenia    NA     NA     NA     NA     NA     NA     NA     NA   1443.  1791.
 7 Aruba      NA     NA     NA     NA   4940.  7390. 10875. 17674. 25121. 26484.
 8 Austra… 10040. 10950. 12217. 14526. 16789. 18334. 19477. 21889. 23425. 26998.
 9 Austria  6137.  8843. 10751. 12835. 16662. 19749. 21597. 23688. 27042. 29096.
10 Azerba…    NA     NA     NA     NA     NA     NA     NA     NA   3456.  1943.
# ℹ 177

You can see that there are many missing observations in the unfiltered data. In real life, you will mostly get unfiltered data -- how should we handle missing data?

## Missing Values
Missing values can be:

* **Explicit** (marked as `NA` in our data); or
* **Implicit** (not present in the data).

In [8]:
df <- tribble(
  ~person,           ~treatment, ~response,
  "Derrick Whitmore", 1,         7,
  NA,                 2,         10,
  NA,                 3,         NA,
  "Katherine Burke",  1,         4
)

In [9]:
df

person,treatment,response
<chr>,<dbl>,<dbl>
Derrick Whitmore,1,7
NA,2,10
NA,3,NA
Katherine Burke,1,4


The missing values are **explicit** in this table: each missing value is indicated by `NA` in the table.

You can fill in these missing values with `tidyr::fill()`. It works like `select()`, taking a set of columns, and fills them in with the last non-missing value.

In [11]:
# fill df for person and response columns
fill(df, person, response)

person,treatment,response
<chr>,<dbl>,<dbl>
Derrick Whitmore,1,7
Derrick Whitmore,2,10
Derrick Whitmore,3,10
Katherine Burke,1,4


In [ ]:
# fill in each missing value in gapminder_unfiltered carrying forward


# A tibble: 3,313 × 6
   country     continent  year lifeExp      pop gdpPercap
   <fct>       <fct>     <int>   <dbl>    <int>     <dbl>
 1 Afghanistan Asia       1952    28.8  8425333      779.
 2 Afghanistan Asia       1957    30.3  9240934      821.
 3 Afghanistan Asia       1962    32.0 10267083      853.
 4 Afghanistan Asia       1967    34.0 11537966      836.
 5 Afghanistan Asia       1972    36.1 13079460      740.
 6 Afghanistan Asia       1977    38.4 14880372      786.
 7 Afghanistan Asia       1982    39.9 12881816      978.
 8 Afghanistan Asia       1987    40.8 13867957      852.
 9 Afghanistan Asia       1992    41.7 16317921      649.
10 Afghanistan Asia       1997    41.8 22227415      635.
# … with 3,303 more rows


## Encoded missing data

Sometimes, data contain a concrete value that actually represents a missing value. You see this often when dealing with data that is imported from a format that does not have a way to represent missing values, such as text or CSV. So it must instead use some special value like 99 or -999.

To correct for this type of missing value, we can use a function called `na_if(x, n)`. This takes a vector `x` and replaces any occurence of `n` with `NA`:

In [ ]:
na_if(c(-99, 1, 3, -99, 2), -99)

[1] NA  1  3 NA  2

## Implicit missing values
A second type of missing data occurs when there are simply no observations in the dataset for a particular combination of columns. For example:

In [ ]:
stocks <- tibble(
  year  = c(2020, 2020, 2020, 2020, 2021, 2021, 2021),
  qtr   = c(   1,    2,    3,    4,    2,    3,    4),
  price = c(1.88, 0.59, 0.35,   NA, 0.92, 0.17, 2.66)
)

In [ ]:
print(stocks)

# A tibble: 7 × 3
   year   qtr price
  <dbl> <dbl> <dbl>
1  2020     1  1.88
2  2020     2  0.59
3  2020     3  0.35
4  2020     4 NA   
5  2021     2  0.92
6  2021     3  0.17
7  2021     4  2.66


This dataset has two missing observations:
- The price in the 2020q4 is explicitly missing. (It has an `NA`.)
- The price in 2021q1 is implicitly missing: it does not appear in the dataset.

> An explicit missing value is the presence of an absence.
>
> An implicit missing value is the absence of a presence.


How can we handle implicit missing values? As we have already seen, one option is to use `pivot_wider()`:

In [ ]:
# using pivot_wider() on stocks converts implicit missing values to explicit
pivot_wider(stocks, names_from = qtr, values_from = price) %>%
  print

# A tibble: 2 × 5
   year   `1`   `2`   `3`   `4`
  <dbl> <dbl> <dbl> <dbl> <dbl>
1  2020  1.88  0.59  0.35 NA   
2  2021 NA     0.92  0.17  2.66


Alternatively, we can use the `complete()` function, which creates entries for all possible combinations of a set of columns:

In [ ]:
# use complete to fill in missing values for stocks
complete(stocks, year, qtr)

year,qtr,price
<dbl>,<dbl>,<dbl>
2020,1,1.88
2020,2,0.59
2020,3,0.35
2020,4,NA
2021,1,NA
2021,2,0.92
2021,3,0.17
2021,4,2.66


### Fixed value replacement

Sometimes missing values represent some fixed and known value, most commonly 0. You can use dplyr::coalesce() to replace them:

In [29]:
x <- c(1, 4, 5, 7, NA)
coalesce(x, 0)
#> [1] 1 4 5 7 0

[1] 1 4 5 7 0

### Titanic analysis

On April 15, 1912, the great "unsinkable" RMS Titanic ship sank to the bottom of the Atlantic.

Unfortunately, there weren’t enough lifeboats for everyone on board, resulting in more than 1500 deaths out of 2224 passengers and crew. Refer:https://en.wikipedia.org/wiki/Titanic

We have partial list of passengers here. Let us do some analysis

In [35]:
titanic = read.csv('https://storage.googleapis.com/mbcc/titanic.csv')
titanic %>% head %>% print

  survived pclass    sex age sibsp parch    fare embarked class   who
1        0      3   male  22     1     0  7.2500        S Third   man
2        1      1 female  38     1     0 71.2833        C First woman
3        1      3 female  26     0     0  7.9250        S Third woman
4        1      1 female  35     1     0 53.1000        S First woman
5        0      3   male  35     0     0  8.0500        S Third   man
6        0      3   male  NA     0     0  8.4583        Q Third   man
  adult_male deck embark_town alive alone
1       True      Southampton    no False
2      False    C   Cherbourg   yes False
3      False      Southampton   yes  True
4      False    C Southampton   yes False
5       True      Southampton    no  True
6       True       Queenstown    no  True


## &#129300; Quiz

What is the average age of the passengers (ignoring the decimal part)?

<ol style="list-style-type: upper-alpha;">
    <li>28</li>
    <li>30</li>
    <li>27</li>
    <li>29</li>
    <li>31</li> 
</ol>


In [37]:
## solution



## &#129300; Quiz

How many na values are there in the age column?

<ol style="list-style-type: upper-alpha;">
    <li>177</li>
    <li>178</li>
    <li>714</li>
    <li>There are no NA values</li>
</ol>


Replace the NA values in the `age` column with average age

In [58]:
## solution
